# Fine-tuning de GPT 3 y ChatGPT

<div style="background-color:#D9EEFF;color:black;padding:2%;">
<h2>Enunciado del caso práctico</h2>

En este caso práctico, se propone al alumno la realización de Fine-tuning sobre GPT-3.5-Turbo (ChatGPT) para la realización de una tarea específica.

Concretamente, se propone un escenario en el que un periódico digital quiere generar automáticamente el título de sus artículos, la descripción y las palabras clave utilizadas para el SEO.

Un factor importante es que la generación de esta información no debe ser genérica, debe adaptarse al estilo y tipo de títulos, descripciones y palabras clave que ha generado el periódico para otros artículos pasados.

</div>

## Referencias importantes
https://platform.openai.com/docs/guides/fine-tuning


# Resolución del caso práctico

## 0. Instalación de librerías externas

In [ ]:
!pip install openai==0.28

## 1. Lectura de la API Key

In [ ]:
import openai

with open("/content/drive/MyDrive/api-keys/secret-key.txt") as f:
  openai.api_key = f.readline()

## 2. Lectura del conjunto de datos de Fine-tuning

Cuando aplicamos Fine-tuning a un modelo de terceros, debemos de asegurarnos de que los datos que vamos a utilizar se encuentran en un formato correcto.

OpenAI proporciona unas guías sobre la elaboración del conjunto de datos: https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset

En términos generales el formato de los ejemplos que debemos utilizar para GPT-3 es el siguiente:
```
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "What's the capital of France?"}, {"role": "assistant", "content": "Paris, as if everyone doesn't know that already."}]}
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "Who wrote 'Romeo and Juliet'?"}, {"role": "assistant", "content": "Oh, just some guy named William Shakespeare. Ever heard of him?"}]}
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "How far is the Moon from Earth?"}, {"role": "assistant", "content": "Around 384,400 kilometers. Give or take a few, like that really matters."}]}
```
El conjunto de datos que vamos a utilizar para este caso práctico esta formado por 10 ejemplos de entrenamiento que tienen el siguiente formato:
```
{"messages":[{"role": "system", "content": "Dado un artículo de noticias, proporciona los siguientes campos en un diccionario JSON: 'titulo', 'SEO' y 'resumen'."},{"role": "user", "content": "Un día después de la cumbre de la Paz en El Cairo, que ha concluido sin una declaración final ante las diferencias de los participantes sobre la guerra de Israel en Gaza, Israel ha anunciado la intensificación de sus ataques en la Franja de Gaza en preparación de una ofensiva terrestre en el enclave palestino."},{"role": "assistant", "content": "{\"titulo\": \"Israel intensifica ataques en Gaza y ataques aéreos en Siria mientras Hamas ofrece liberación de rehenes\", \"SEO\": [\"Israel, Gaza, ofensiva terrestre, ataque aéreo, Franja de Gaza, Siria, ataques simultáneos, Hamas, Estados Unidos, Irán\"], \"resumen\": \"Israel intensifica ataques en Gaza y Siria. Hamas ofrece liberación de rehenes. Escalada en el conflicto en Oriente Medio.\"}"}]}
```


---

**IMPORTANTE**

Revisa el siguiente Google Colab para saber el coste que tendrá el entrenamiento y si el formato de tus datos es el adecuado:
https://colab.research.google.com/drive/1MgPDXMxA5F3g2D8gDwGVCTnEIdQuQPlU#scrollTo=c248ccd1

---

In [ ]:
# Lectura del conjunto de datos
openai.File.create(
  file=open("/content/drive/MyDrive/datasets/fine-tuning-dataset.jsonl", "rb"),
  purpose='fine-tune'
)

## 3. Fine-tuning de GPT-3.5-turbo y ChatGPT

OpenAI nos permite modificar las siguientes opciones o hiperparámetros de entrenamiento:

* `n_epochs`: Este hiperparámetro se refiere al número de veces que todo el conjunto de datos se pasa a través del modelo de aprendizaje automático durante el proceso de entrenamiento.

* `batch_size`: El tamaño de lote se refiere al número de ejemplos de entrenamiento que se procesan simultáneamente en cada iteración durante el entrenamiento del modelo.

* `learning_rate_multiplier`: La tasa de aprendizaje es un hiperparámetro crítico en el entrenamiento de modelos de aprendizaje automático. Controla el tamaño de los ajustes que se realizan en los parámetros del modelo durante el entrenamiento.

In [ ]:
# Definicion de los hiperparametros
hyperparameters = {"n_epochs":3}

Además del conjunto de datos de entrenamiento, lo ideal es proporcionarle un conjunto de datos de validación (`validation_file`) para que calcule métricas durante el entrenamiento.

In [ ]:
# Fine-tuning del modelo
openai.FineTuningJob.create(
    training_file="file-DuUyCs765h7hdrIkRutTvwDE", # Debe indicarse el id obtenido en la seccion anterior
    model="gpt-3.5-turbo",
    hyperparameters=hyperparameters)

## 4. Monitorización del Fine-tuning

### Listar los procesos de fine-tuning en curso

In [ ]:
openai.FineTuningJob.list(limit=10)

### Obtener el estado de un proceso de fine-tuning

In [ ]:
FT_JOB_ID = "ftjob-VUtzVtVpEQfg9wX8fELPH3BS"

In [ ]:
openai.FineTuningJob.retrieve(FT_JOB_ID)

### Listar los eventos de un proceso de fine-tuning

In [ ]:
openai.FineTuningJob.list_events(id=FT_JOB_ID, limit=10)

## 5. Generación de texto con GPT Fine-tuned

In [ ]:
def obtener_completion(prompt, model="ft:gpt-3.5-turbo-0613:personal::8ClqSaNA"):
  mensaje = [
      {"role": "system", "content": "Dado un artículo de noticias, proporciona los siguientes campos en un diccionario JSON: 'titulo', 'SEO' y 'resumen'."},
      {"role": "user", "content": prompt}
  ]
  respuesta = openai.ChatCompletion.create(
      model=model,
      messages=mensaje,
      temperature=0, # Este hiperparámetro controla la aleatoriedad del modelo
  )
  return respuesta.choices[0].message["content"]

In [ ]:
prompt = """La tecnología nos ha brindado la oportunidad de tener en nuestro \
bolsillo un aparato que es a la vez agenda y enciclopedia, que nos recuerda los \
cumpleaños de nuestras amistades y nos da acceso a todos los detalles de la tabla \
periódica. Aún así, tener una buena memoria nos ayuda en numerosos aspectos de \
nuestra vida.

El problema es que, ya sea porque somos estudiantes y tenemos que memorizar \
cantidad de conceptos o porque nos hacemos mayores y nuestro cerebro pierde \
capacidades, son muchas las razones para tratar de buscar herramientas que nos \
permitan entrenar y mantener nuestra capacidad mnemotécnica. Estas son algunas \
de las estrategias que han sido validadas por estudios científicos.

La primera nos recuerda aquella frase latina de “mens sana in corpore sano”, \
extraída de las Sátiras del autor romano Décimo Junio Juvenal. Distintos \
organismos médicos, como los Centros para el Control y Prevención de Enfermedades \
de EE UU o la Organización Mundial de la Salud, vinculan el ejercicio físico con \
la buena memoria.

El ejercicio podría ser especialmente recomendable a la hora de evitar la \
degeneración de nuestra memoria vinculada con la edad y con enfermedades como \
el Alzheimer. Este mismo año pueden encontrarse dos estudios centrados en esta \
cuestión, uno publicado en Alzheimer's Research & Therapy, y otro en Journal of \
Neurology, Neurosurgery & Psychiatry.

No sabemos exactamente el porqué de la relación. Mientras algunos sugieren que \
puede deberse a la proteína BNDF, otros creen que la relación puede originarse \
en una mejor oxigenación del cerebro. Más estudios se requieren en este sentido, \
también para establecer hasta qué punto la relación puede extrapolarse a personas \
más jóvenes. El problema está en que muchos de los estudios realizados en este \
contexto tienen muestras limitadas, lo que limita la posibilidad de generalizar \
sus resultados."""

In [ ]:
obtener_completion(prompt)

### ¿Cómo se lo presentamos a los usuarios?

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Crear un widget de área de texto
text_area = widgets.Textarea(
    value='Introduce el texto del artículo...',
    description='',
    disabled=False,
    layout=widgets.Layout(width='70%', height='300px')  # Ajusta el ancho y la altura según tus preferencias
)

# Crear un botón para aceptar/enviar
submit_button = widgets.Button(description='Enviar')

# Función que se ejecutará cuando se haga clic en el botón
def handle_submit_button_click(b):
    article_text = text_area.value
    completion = obtener_completion(article_text)
    resultados = eval(completion)
    print(f"\n\nResultado del análisis:\nTitulo: {resultados['titulo']}\nSEO: {resultados['SEO']}\nResumen: {resultados['resumen']}")

# Asociar la función con el evento de hacer clic en el botón
submit_button.on_click(handle_submit_button_click)

# Mostrar el widget de área de texto y el botón en la celda de Colab
display(text_area)
display(submit_button)
